In [2]:
import sqlite3
conn = sqlite3.connect('w1m2.db')
cursor = conn.cursor()

In [3]:

cursor.execute(
    '''
    CREATE TABLE Customers 
    (CustomerID integer, CustomerName text, ContactName text, Address text, 
    City text, PostalCode text, Country text) 
    '''
)

cursor.execute(
    '''
    CREATE TABLE Categories
    (CategoryID integer, CategoryName text, Description text) 
    '''
)

cursor.execute(
    '''
    CREATE TABLE Employees
    (EmployeeID integer, LastName text, FirstName text, BirthDate text, 
    Photo text, Notes text) 
    '''
)

cursor.execute(
    '''
    CREATE TABLE OrderDetails
    (CategoryID integer, OrderID integer, ProductID integer, 
    Quantity integer) 
    '''
)

cursor.execute(
    '''
    CREATE TABLE Orders
    (OrderID integer, CustomerID integer, EmployeeID integer, 
    OrderDate text, ShipperID integer) 
    '''
)

cursor.execute(
    '''
    CREATE TABLE Products
    (ProductID integer, ProductName text, SupplierID integer, 
    CategoryID integer, Unit text, Price real) 
    '''
)

cursor.execute(
    '''
    CREATE TABLE Shippers
    (ShipperID integer, ShipperName text, Phone text) 
    '''
)

cursor.execute(
    '''
    CREATE TABLE Suppliers
    (SupplierID integer, SupplierName text, ContactName text,
    Address text, City text, PostalCode text, Country text, Phone text) 
    '''
)

conn.commit()




In [ ]:
import csv
import os

folder_path = './data/m2_csv'

for csv_file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, csv_file)
    file_name_without_ext, file_ext = os.path.splitext(csv_file)
    if file_ext != '.csv':
        continue
    table_name = file_name_without_ext.capitalize()
    cursor.execute(f"PRAGMA table_info({table_name})")
    schema = {col[1]: col[2].upper() for col in cursor.fetchall()}
    column_names = list(schema.keys())
    with open(file_path, 'r', encoding='utf-8') as f:
        
        reader = csv.DictReader(f)
        
        rows_to_insert = []
        for row in reader:
            converted_row = []
            for col in column_names:
                val = row[col]
                col_type = schema[col]

                if col_type == "TEXT":
                    converted_row.append(str(val))
                elif col_type == "INTEGER":
                    converted_row.append(int(val) if val.isdigit() else None)
                elif col_type == "REAL":
                    converted_row.append(float(val))
                else:
                    converted_row.append(val)

            rows_to_insert.append(tuple(converted_row))

    placeholders = ', '.join(['?'] * len(column_names))
    cursor.executemany(f"INSERT INTO {table_name} VALUES ({placeholders})", rows_to_insert)

conn.commit()



In [5]:
from tabulate import tabulate
cursor.execute(
    '''
    SELECT * FROM Customers;
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+--------------------------------------+----------------------+------------------------------------------------+-----------------+--------------+-------------+
|   CustomerID | CustomerName                         | ContactName          | Address                                        | City            | PostalCode   | Country     |
+==============+======================================+======================+================================================+=================+==============+=============+
|            1 | Alfreds Futterkiste                  | Maria Anders         | Obere Str. 57                                  | Berlin          | 12209        | Germany     |
+--------------+--------------------------------------+----------------------+------------------------------------------------+-----------------+--------------+-------------+
|            2 | Ana Trujillo Emparedados y helados   | Ana Trujillo         | Avda. de la Constitución 2222                 

In [6]:
cursor.execute(
    '''
    SELECT CustomerName, City FROM Customers;
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------------------------------+-----------------+
| CustomerName                         | City            |
+======================================+=================+
| Alfreds Futterkiste                  | Berlin          |
+--------------------------------------+-----------------+
| Ana Trujillo Emparedados y helados   | México D.F.     |
+--------------------------------------+-----------------+
| Antonio Moreno Taquería              | México D.F.     |
+--------------------------------------+-----------------+
| Around the Horn                      | London          |
+--------------------------------------+-----------------+
| Berglunds snabbköp                   | Luleå           |
+--------------------------------------+-----------------+
| Blauesr See Delikatessen             | Mannheim        |
+--------------------------------------+-----------------+
| Blondel père et fils                 | Strasbourg      |
+--------------------------------------+----------------

In [7]:
cursor.execute(
    '''
    SELECT DISTINCT Country FROM Customers;
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+-------------+
| Country     |
+=============+
| Germany     |
+-------------+
| Mexico      |
+-------------+
| UK          |
+-------------+
| Sweden      |
+-------------+
| France      |
+-------------+
| Spain       |
+-------------+
| Canada      |
+-------------+
| Argentina   |
+-------------+
| Switzerland |
+-------------+
| Brazil      |
+-------------+
| Austria     |
+-------------+
| Italy       |
+-------------+
| Portugal    |
+-------------+
| USA         |
+-------------+
| Venezuela   |
+-------------+
| Ireland     |
+-------------+
| Belgium     |
+-------------+
| Norway      |
+-------------+
| Denmark     |
+-------------+
| Finland     |
+-------------+
| Poland      |
+-------------+


In [13]:
cursor.execute(
    '''
    SELECT COUNT(DISTINCT Country) FROM Customers;
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+---------------------------+
|   COUNT(DISTINCT Country) |
+===========================+
|                        21 |
+---------------------------+


In [14]:
cursor.execute(
    '''
    SELECT Count(*) AS DistinctCountries
    FROM (SELECT DISTINCT Country FROM Customers);
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+---------------------+
|   DistinctCountries |
+=====================+
|                  21 |
+---------------------+


In [8]:
cursor.execute(
    '''
    SELECT * FROM Customers
    WHERE Country='Mexico';
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+------------------------------------+----------------------+-------------------------------+-------------+--------------+-----------+
|   CustomerID | CustomerName                       | ContactName          | Address                       | City        |   PostalCode | Country   |
+==============+====================================+======================+===============================+=============+==============+===========+
|            2 | Ana Trujillo Emparedados y helados | Ana Trujillo         | Avda. de la Constitución 2222 | México D.F. |        05021 | Mexico    |
+--------------+------------------------------------+----------------------+-------------------------------+-------------+--------------+-----------+
|            3 | Antonio Moreno Taquería            | Antonio Moreno       | Mataderos 2312                | México D.F. |        05023 | Mexico    |
+--------------+------------------------------------+----------------------+------------------------

In [15]:
cursor.execute(
    '''
    SELECT * FROM Customers
    WHERE CustomerID=1;
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+---------------------+---------------+---------------+--------+--------------+-----------+
|   CustomerID | CustomerName        | ContactName   | Address       | City   |   PostalCode | Country   |
+==============+=====================+===============+===============+========+==============+===========+
|            1 | Alfreds Futterkiste | Maria Anders  | Obere Str. 57 | Berlin |        12209 | Germany   |
+--------------+---------------------+---------------+---------------+--------+--------------+-----------+


In [16]:
cursor.execute(
    '''
    SELECT * FROM Customers
    WHERE CustomerID > 80;
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+-----------------------------------+-------------------+-----------------------------+-----------+--------------+-----------+
|   CustomerID | CustomerName                      | ContactName       | Address                     | City      | PostalCode   | Country   |
+==============+===================================+===================+=============================+===========+==============+===========+
|           81 | Tradição Hipermercados            | Anabela Domingues | Av. Inês de Castro, 414     | São Paulo | 05634-030    | Brazil    |
+--------------+-----------------------------------+-------------------+-----------------------------+-----------+--------------+-----------+
|           82 | Trail's Head Gourmet Provisioners | Helvetius Nagy    | 722 DaVinci Blvd.           | Kirkland  | 98034        | USA       |
+--------------+-----------------------------------+-------------------+-----------------------------+-----------+--------------+-----------+
|     

In [9]:
cursor.execute(
    '''
    SELECT * FROM Products ORDER BY Price;
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+-------------+----------------------------------+--------------+--------------+----------------------+---------+
|   ProductID | ProductName                      |   SupplierID |   CategoryID | Unit                 |   Price |
+=============+==================================+==============+==============+======================+=========+
|          33 | Geitost                          |           15 |            4 | 500 g                |    2.5  |
+-------------+----------------------------------+--------------+--------------+----------------------+---------+
|          24 | Guaraná Fantástica               |           10 |            1 | 12 - 355 ml cans     |    4.5  |
+-------------+----------------------------------+--------------+--------------+----------------------+---------+
|          13 | Konbu                            |            6 |            8 | 2 kg box             |    6    |
+-------------+----------------------------------+--------------+--------------+--------

In [17]:
cursor.execute(
    '''
    SELECT * FROM Products ORDER BY Price DESC;
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+-------------+----------------------------------+--------------+--------------+----------------------+---------+
|   ProductID | ProductName                      |   SupplierID |   CategoryID | Unit                 |   Price |
+=============+==================================+==============+==============+======================+=========+
|          38 | Côte de Blaye                    |           18 |            1 | 12 - 75 cl bottles   |  263.5  |
+-------------+----------------------------------+--------------+--------------+----------------------+---------+
|          29 | Thüringer Rostbratwurst          |           12 |            6 | 50 bags x 30 sausgs. |  123.79 |
+-------------+----------------------------------+--------------+--------------+----------------------+---------+
|           9 | Mishi Kobe Niku                  |            4 |            6 | 18 - 500 g pkgs.     |   97    |
+-------------+----------------------------------+--------------+--------------+--------

In [18]:
cursor.execute(
    '''
    SELECT * FROM Products ORDER BY ProductName;
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+-------------+----------------------------------+--------------+--------------+----------------------+---------+
|   ProductID | ProductName                      |   SupplierID |   CategoryID | Unit                 |   Price |
+=============+==================================+==============+==============+======================+=========+
|          17 | Alice Mutton                     |            7 |            6 | 20 - 1 kg tins       |   39    |
+-------------+----------------------------------+--------------+--------------+----------------------+---------+
|           3 | Aniseed Syrup                    |            1 |            2 | 12 - 550 ml bottles  |   10    |
+-------------+----------------------------------+--------------+--------------+----------------------+---------+
|          40 | Boston Crab Meat                 |           19 |            8 | 24 - 4 oz tins       |   18.4  |
+-------------+----------------------------------+--------------+--------------+--------

In [19]:
cursor.execute(
    '''
    SELECT * FROM Products ORDER BY ProductName DESC;
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+-------------+----------------------------------+--------------+--------------+----------------------+---------+
|   ProductID | ProductName                      |   SupplierID |   CategoryID | Unit                 |   Price |
+=============+==================================+==============+==============+======================+=========+
|          47 | Zaanse koeken                    |           22 |            3 | 10 - 4 oz boxes      |    9.5  |
+-------------+----------------------------------+--------------+--------------+----------------------+---------+
|          64 | Wimmers gute Semmelknödel        |           12 |            5 | 20 bags x 4 pieces   |   33.25 |
+-------------+----------------------------------+--------------+--------------+----------------------+---------+
|          63 | Vegie-spread                     |            7 |            2 | 15 - 625 g jars      |   43.9  |
+-------------+----------------------------------+--------------+--------------+--------

In [20]:
cursor.execute(
    '''
    SELECT * FROM Customers ORDER BY Country, CustomerName;
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+--------------------------------------+----------------------+------------------------------------------------+-----------------+--------------+-------------+
|   CustomerID | CustomerName                         | ContactName          | Address                                        | City            | PostalCode   | Country     |
+==============+======================================+======================+================================================+=================+==============+=============+
|           12 | Cactus Comidas para llevar           | Patricio Simpson     | Cerrito 333                                    | Buenos Aires    | 1010         | Argentina   |
+--------------+--------------------------------------+----------------------+------------------------------------------------+-----------------+--------------+-------------+
|           54 | Océano Atlántico Ltda.               | Yvonne Moncada       | Ing. Gustavo Moncada 8585 Piso 20-A           

In [21]:
cursor.execute(
    '''
    SELECT * FROM Customers
    ORDER BY Country ASC, CustomerName DESC;
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+--------------------------------------+----------------------+------------------------------------------------+-----------------+--------------+-------------+
|   CustomerID | CustomerName                         | ContactName          | Address                                        | City            | PostalCode   | Country     |
+==============+======================================+======================+================================================+=================+==============+=============+
|           64 | Rancho grande                        | Sergio Gutiérrez     | Av. del Libertador 900                         | Buenos Aires    | 1010         | Argentina   |
+--------------+--------------------------------------+----------------------+------------------------------------------------+-----------------+--------------+-------------+
|           54 | Océano Atlántico Ltda.               | Yvonne Moncada       | Ing. Gustavo Moncada 8585 Piso 20-A           

In [10]:
cursor.execute(
    '''
    SELECT * FROM Customers
    WHERE Country = 'Spain' AND CustomerName LIKE 'G%';
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+------------------------+-------------------+------------------------+-----------+--------------+-----------+
|   CustomerID | CustomerName           | ContactName       | Address                | City      |   PostalCode | Country   |
+==============+========================+===================+========================+===========+==============+===========+
|           29 | Galería del gastrónomo | Eduardo Saavedra  | Rambla de Cataluña, 23 | Barcelona |        08022 | Spain     |
+--------------+------------------------+-------------------+------------------------+-----------+--------------+-----------+
|           30 | Godos Cocina Típica    | José Pedro Freyre | C/ Romero, 33          | Sevilla   |        41101 | Spain     |
+--------------+------------------------+-------------------+------------------------+-----------+--------------+-----------+


In [24]:
cursor.execute(
    '''
    SELECT * FROM Customers
    WHERE Country = 'Brazil'
    AND City = 'Rio de Janeiro'
    AND CustomerID > 50;
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+--------------------+------------------+-------------------------+----------------+--------------+-----------+
|   CustomerID | CustomerName       | ContactName      | Address                 | City           | PostalCode   | Country   |
+==============+====================+==================+=========================+================+==============+===========+
|           61 | Que Delícia        | Bernardo Batista | Rua da Panificadora, 12 | Rio de Janeiro | 02389-673    | Brazil    |
+--------------+--------------------+------------------+-------------------------+----------------+--------------+-----------+
|           67 | Ricardo Adocicados | Janete Limeira   | Av. Copacabana, 267     | Rio de Janeiro | 02389-890    | Brazil    |
+--------------+--------------------+------------------+-------------------------+----------------+--------------+-----------+


In [22]:
cursor.execute(
    '''
    SELECT * FROM Customers
    WHERE Country = 'Spain' AND (CustomerName LIKE 'G%' OR CustomerName LIKE 'R%');
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+------------------------+-------------------+------------------------+-----------+--------------+-----------+
|   CustomerID | CustomerName           | ContactName       | Address                | City      |   PostalCode | Country   |
+==============+========================+===================+========================+===========+==============+===========+
|           29 | Galería del gastrónomo | Eduardo Saavedra  | Rambla de Cataluña, 23 | Barcelona |        08022 | Spain     |
+--------------+------------------------+-------------------+------------------------+-----------+--------------+-----------+
|           30 | Godos Cocina Típica    | José Pedro Freyre | C/ Romero, 33          | Sevilla   |        41101 | Spain     |
+--------------+------------------------+-------------------+------------------------+-----------+--------------+-----------+
|           69 | Romero y tomillo       | Alejandra Camino  | Gran Vía, 1            | Madrid    |        28001 | Spai

In [23]:
cursor.execute(
    '''
    SELECT * FROM Customers
    WHERE Country = 'Spain' AND CustomerName LIKE 'G%' OR CustomerName LIKE 'R%'
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+----------------------------+-------------------+------------------------+----------------+--------------+-------------+
|   CustomerID | CustomerName               | ContactName       | Address                | City           | PostalCode   | Country     |
+==============+============================+===================+========================+================+==============+=============+
|           29 | Galería del gastrónomo     | Eduardo Saavedra  | Rambla de Cataluña, 23 | Barcelona      | 08022        | Spain       |
+--------------+----------------------------+-------------------+------------------------+----------------+--------------+-------------+
|           30 | Godos Cocina Típica        | José Pedro Freyre | C/ Romero, 33          | Sevilla        | 41101        | Spain       |
+--------------+----------------------------+-------------------+------------------------+----------------+--------------+-------------+
|           64 | Rancho grande           

In [25]:
cursor.execute(
    '''
    SELECT *
    FROM Customers
    WHERE Country = 'Germany' OR Country = 'Spain';
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+--------------------------------------+---------------------+------------------------+----------------+--------------+-----------+
|   CustomerID | CustomerName                         | ContactName         | Address                | City           |   PostalCode | Country   |
+==============+======================================+=====================+========================+================+==============+===========+
|            1 | Alfreds Futterkiste                  | Maria Anders        | Obere Str. 57          | Berlin         |        12209 | Germany   |
+--------------+--------------------------------------+---------------------+------------------------+----------------+--------------+-----------+
|            6 | Blauesr See Delikatessen             | Hanna Moos          | Forsterstr. 57         | Mannheim       |        68306 | Germany   |
+--------------+--------------------------------------+---------------------+------------------------+----------------

In [26]:
cursor.execute(
    '''
    SELECT * FROM Customers
    WHERE City = 'Berlin' OR CustomerName LIKE 'G%' OR Country = 'Norway';
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+-------------------------+-------------------+---------------------------+-----------+--------------+-----------+
|   CustomerID | CustomerName            | ContactName       | Address                   | City      | PostalCode   | Country   |
+==============+=========================+===================+===========================+===========+==============+===========+
|            1 | Alfreds Futterkiste     | Maria Anders      | Obere Str. 57             | Berlin    | 12209        | Germany   |
+--------------+-------------------------+-------------------+---------------------------+-----------+--------------+-----------+
|           29 | Galería del gastrónomo  | Eduardo Saavedra  | Rambla de Cataluña, 23    | Barcelona | 08022        | Spain     |
+--------------+-------------------------+-------------------+---------------------------+-----------+--------------+-----------+
|           30 | Godos Cocina Típica     | José Pedro Freyre | C/ Romero, 33             |

In [12]:
cursor.execute(
    '''
    SELECT * FROM Customers
    WHERE NOT Country = 'Spain';
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+------------------------------------+----------------------+------------------------------------------------+-----------------+--------------+-------------+
|   CustomerID | CustomerName                       | ContactName          | Address                                        | City            | PostalCode   | Country     |
+==============+====================================+======================+================================================+=================+==============+=============+
|            1 | Alfreds Futterkiste                | Maria Anders         | Obere Str. 57                                  | Berlin          | 12209        | Germany     |
+--------------+------------------------------------+----------------------+------------------------------------------------+-----------------+--------------+-------------+
|            2 | Ana Trujillo Emparedados y helados | Ana Trujillo         | Avda. de la Constitución 2222                  | México D.

In [27]:
cursor.execute(
    '''
    SELECT * FROM Customers
    WHERE CustomerName NOT LIKE 'A%';
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+--------------------------------------+----------------------+------------------------------------------------+-----------------+--------------+-------------+
|   CustomerID | CustomerName                         | ContactName          | Address                                        | City            | PostalCode   | Country     |
+==============+======================================+======================+================================================+=================+==============+=============+
|            5 | Berglunds snabbköp                   | Christina Berglund   | Berguvsvägen 8                                 | Luleå           | S-958 22     | Sweden      |
+--------------+--------------------------------------+----------------------+------------------------------------------------+-----------------+--------------+-------------+
|            6 | Blauesr See Delikatessen             | Hanna Moos           | Forsterstr. 57                                

In [28]:
cursor.execute(
    '''
    SELECT * FROM Customers
    WHERE CustomerID NOT BETWEEN 10 AND 60;
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+------------------------------------+----------------------+-------------------------------+----------------+--------------+-------------+
|   CustomerID | CustomerName                       | ContactName          | Address                       | City           | PostalCode   | Country     |
+==============+====================================+======================+===============================+================+==============+=============+
|            1 | Alfreds Futterkiste                | Maria Anders         | Obere Str. 57                 | Berlin         | 12209        | Germany     |
+--------------+------------------------------------+----------------------+-------------------------------+----------------+--------------+-------------+
|            2 | Ana Trujillo Emparedados y helados | Ana Trujillo         | Avda. de la Constitución 2222 | México D.F.    | 05021        | Mexico      |
+--------------+------------------------------------+-----------------

In [30]:
cursor.execute(
    '''
    SELECT * FROM Customers
    WHERE City NOT IN ('Paris', 'London');
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+--------------------------------------+----------------------+------------------------------------------------+-----------------+--------------+-------------+
|   CustomerID | CustomerName                         | ContactName          | Address                                        | City            | PostalCode   | Country     |
+==============+======================================+======================+================================================+=================+==============+=============+
|            1 | Alfreds Futterkiste                  | Maria Anders         | Obere Str. 57                                  | Berlin          | 12209        | Germany     |
+--------------+--------------------------------------+----------------------+------------------------------------------------+-----------------+--------------+-------------+
|            2 | Ana Trujillo Emparedados y helados   | Ana Trujillo         | Avda. de la Constitución 2222                 

In [31]:
cursor.execute(
    '''
    SELECT * FROM Customers
    WHERE NOT CustomerID > 50;
    '''
)
headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))

+--------------+--------------------------------------+--------------------+------------------------------------------------+-----------------+--------------+-------------+
|   CustomerID | CustomerName                         | ContactName        | Address                                        | City            | PostalCode   | Country     |
+==============+======================================+====================+================================================+=================+==============+=============+
|            1 | Alfreds Futterkiste                  | Maria Anders       | Obere Str. 57                                  | Berlin          | 12209        | Germany     |
+--------------+--------------------------------------+--------------------+------------------------------------------------+-----------------+--------------+-------------+
|            2 | Ana Trujillo Emparedados y helados   | Ana Trujillo       | Avda. de la Constitución 2222                  | México D.

In [ ]:
cursor.execute(
    '''
    INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
    VALUES (92, 'Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway');
    '''
)
conn.commit()

cursor.execute(
    '''
    select * from customers;
    '''
)

headers = [description[0] for description in cursor.description]
rows = cursor.fetchall()
print(tabulate(rows, headers=headers, tablefmt='grid'))


+--------------+--------------------------------------+----------------------+------------------------------------------------+-----------------+--------------+-------------+
|   CustomerID | CustomerName                         | ContactName          | Address                                        | City            | PostalCode   | Country     |
+==============+======================================+======================+================================================+=================+==============+=============+
|            1 | Alfreds Futterkiste                  | Maria Anders         | Obere Str. 57                                  | Berlin          | 12209        | Germany     |
+--------------+--------------------------------------+----------------------+------------------------------------------------+-----------------+--------------+-------------+
|            2 | Ana Trujillo Emparedados y helados   | Ana Trujillo         | Avda. de la Constitución 2222                 